In [1]:
import os
import pdb
import torch
import pandas
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
import torch.nn.functional as F

In [2]:
dataset = Planetoid(root='data/Planetoid', name='Pubmed', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: Pubmed():
Number of graphs: 1
Number of features: 500
Number of classes: 3
Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [3]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        
        self.conv1 = GATConv(dataset.num_features, hidden_channels, dropout=0.6)
        self.conv2 = GATConv(hidden_channels, dataset.num_classes, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
    acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
    return acc

for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GAT(
  (conv1): GATConv(500, 8, heads=1)
  (conv2): GATConv(8, 3, heads=1)
)
Epoch: 001, Loss: 1.1000, Val: 0.4020, Test: 0.3560
Epoch: 002, Loss: 1.0942, Val: 0.4320, Test: 0.4100
Epoch: 003, Loss: 1.0920, Val: 0.5560, Test: 0.5220
Epoch: 004, Loss: 1.0928, Val: 0.6620, Test: 0.6230
Epoch: 005, Loss: 1.0876, Val: 0.6440, Test: 0.6390
Epoch: 006, Loss: 1.0846, Val: 0.6660, Test: 0.6520
Epoch: 007, Loss: 1.0934, Val: 0.6540, Test: 0.6230
Epoch: 008, Loss: 1.0779, Val: 0.6500, Test: 0.6060
Epoch: 009, Loss: 1.0811, Val: 0.6040, Test: 0.5770
Epoch: 010, Loss: 1.0795, Val: 0.5720, Test: 0.5650
Epoch: 011, Loss: 1.0738, Val: 0.5560, Test: 0.5600
Epoch: 012, Loss: 1.0869, Val: 0.5720, Test: 0.5710
Epoch: 013, Loss: 1.0796, Val: 0.6600, Test: 0.6090
Epoch: 014, Loss: 1.0699, Val: 0.6840, Test: 0.6530
Epoch: 015, Loss: 1.0579, Val: 0.7260, Test: 0.7110
Epoch: 016, Loss: 1.0632, Val: 0.7460, Test: 0.7020
Epoch: 017, Loss: 1.0474, Val: 0.7480, Test: 0.7050
Epoch: 018, Loss: 1.0547, Val: 0.7420, 

In [5]:
test_acc = test(data.val_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7520
